In [36]:
!pip install tensorflow opencv-python matplotlib


In [37]:
!pip install ultralytics


In [39]:
import zipfile
import os

with zipfile.ZipFile("Garbage image dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset")


In [41]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 80-20 split
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

train_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(128,128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(128,128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 2024 images belonging to 6 classes.
Found 503 images belonging to 6 classes.


In [42]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128,128,3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(len(train_gen.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


for layer in base_model.layers:
    layer.trainable = False


model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


history = model.fit(train_gen, validation_data=val_gen, epochs=10)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 58s 743ms/step - accuracy: 0.2646 - loss: 1.8235 - val_accuracy: 0.5268 - val_loss: 1.2359
Epoch 2/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 46s 726ms/step - accuracy: 0.6013 - loss: 1.1048 - val_accuracy: 0.6501 - val_loss: 0.9617
Epoch 3/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 45s 700ms/step - accuracy: 0.6817 - loss: 0.8864 - val_accuracy: 0.7038 - val_loss: 0.8812
Epoch 4/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 49s 761ms/step - accuracy: 0.7230 - loss: 0.7721 - val_accuracy: 0.7276 - val_loss: 0.7987
Epoch 5/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 47s 735ms/step - accuracy: 0.7481 - loss: 0.6892 - val_accuracy: 0.7237 - val_loss: 0.7795
Epoch 6/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 46s 715ms/step - accuracy: 0.7986 - loss: 0.6010 - val_accuracy: 0.7177 - val_loss: 0.7659
Epoch 7/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 45s 702ms/step - accuracy: 0.8051 - loss: 0.5590 - val_accuracy: 0.7614 - val_loss: 0.7084
Epoch 8/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 46s 713ms/step - accuracy: 0.8194 - loss: 0.5483 - val_accu

In [43]:
model.save("scrap_model.h5")
print("Model saved successfully!")


Model saved successfully!


In [44]:
import cv2
from google.colab.patches import cv2_imshow
import glob
import numpy as np
import json
import time
from tensorflow.keras.models import load_model

model = load_model("scrap_model.h5")

class_names = list(train_gen.class_indices.keys())

object_counter = {name: 0 for name in class_names}

dataset_path = "dataset/TrashType_Image_Dataset"
image_paths = glob.glob(dataset_path + "/*/*.jpg")
image_paths.sort()

for path in image_paths[:50]:
    img = cv2.imread(path)
    if img is None:
        continue

    img_resized = cv2.resize(img, (128, 128)) / 255.0
    img_input = np.expand_dims(img_resized, axis=0)

    start_time = time.time()

    predictions = model.predict(img_input)

    end_time = time.time()
    latency = (end_time - start_time) * 1000


    pred_index = np.argmax(predictions)
    predicted_label = class_names[pred_index]
    object_counter[predicted_label] += 1


    height, width, _ = img.shape
    pick_point = (width // 2, height // 2)
    cv2.circle(img, pick_point, 10, (0, 255, 0), -1)

    cv2.putText(img, f"Type: {predicted_label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(img, f"Latency: {latency:.1f} ms", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
    cv2.putText(img, f"Counts: {object_counter}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

    ros_msg = {
        "label": predicted_label,
        "confidence": float(predictions[0][pred_index]),
        "pick_point": {"x": int(pick_point[0]), "y": int(pick_point[1])},
        "timestamp": time.strftime("%H:%M:%S")
    }
    print(json.dumps(ros_msg))


    cv2_imshow(img)
    cv2.waitKey(200)


Output hidden; open in https://colab.research.google.com to view.

In [45]:
out = cv2.VideoWriter('scrap_simulation.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 5, (w,h))

for img_path in image_files[:50]:

    out.write(frame)

out.release()


In [48]:
from google.colab import files
files.download("/content/scrap_model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>